Contamos con un cluster que tiene 4 computadoras. Queremos
aprovechar el paralelismo del cluster para calcular los números primos
entre 2 y 20.000.000. Para esto usaremos el conocido algoritmo de la
criba de Eratóstenes. Por ejemplo si empezamos con una lista de tipo
(2,3,4,5,6,7,8...) en un primer paso eliminamos todos los que son
mayores a 2 y divisibles por 2 y nos queda (2,3,5,7,9,11,13…) luego
eliminamos todos los mayores a 3 divisibles por 3 y nos queda
(2,3,5,7,11,13….etc) luego todos los divisibles por 5 y así
sucesivamente. El resultado final es una lista de números que son
primos.  
[Link](https://piazza.com/class_profile/get_resource/jkr2voxi1yw4wt/jkr2vqs33v4xw)

En primer lugar vamos a explicar la idea: inicialmente tenemos todos los numeros entre 2 y 20M. En cada paso, vamos a tomar el menor de los numeros que no hayamos revisado (empezando desde el 2) y _tachando_ aquellos divisibles por ese numero.

In [ ]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
import math
from functools import partial
maxVal = int(1e2)
rdd = sc.parallelize(range(2,maxVal+1))

# Función para aplicar parcialmente
def _isDivisible(n,div):
    return n==div or n%div != 0

for div in range(2,int(math.sqrt(maxVal+1))):
    isDivisible = partial(_isDivisible,div=div)
    rdd = rdd.filter(isDivisible)

rdd.collect()

La criba de Eratostenes tiene una complejidad algoritmica de O(N log(log N)), y permite ciertas optimizaciones, pero para mantenerlo simple, dejaremos esta version sencilla.

Inicialmente intenté usar una lambda que tomara el valor de div, pero no funciona. Para mas detalles, pueden leer [esto](https://stackoverflow.com/a/38085794/4864169) y [esto](https://stackoverflow.com/a/34021333/4864169). Mas alla de ese detalle, lo importante es entender lo siguiente:

Si un numero $n$ no es primo, puede descomponerse en dos numeros $a, b$ tales que $a * b = n$. Si tanto $a$ como $b$ fueran mas grandes que $\sqrt n$ entonces $a * b > n$, lo cual es un absurdo bajo las hipotesis anteriores. Por lo tanto, chequeando la divisibilidad de un numero $n$ contra todos los numeros $m < \sqrt n$, podemos asegurar si un número es primo o no.

Entonces hacemos $\sqrt n$ iteraciones y en cada una filtramos el RDD por el numero de iteración $n$, quedandonos solo con $\{\,x \in \text{ RDD } \mid x = n \vee x \ncong n \}$.